In [3]:
from __future__ import print_function
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Download and create dataframes 

The *ml100k* datasets containts different items for different usecases. We are only interested in the .user and .data items.

In [0]:
# Read the users and ratings datasets

users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-100k/u.user', sep='|', names=users_cols)

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

In [0]:
# Read in the movies datasets

genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
    
movies_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] + genre_cols
movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')
movies = movies.drop(columns=['release_date', 'video_release_date', 'imdb_url'])

In [0]:
# Split the data

def split_dataframe(df, holdout_fraction=0.1):
    
  test = df.sample(frac=0.1, replace=False)
  train = df[~df.index.isin(test.index)]

  return train, test

# Matrix Factorization

The goal of matrix factorization is to decompose the feedback matrix, $A_{nm}$ into a product approximation of 2 embedding vector that is $U_n$ and $V_m$. 

To put it in simple terms:

$A_{NM}$ approximates
$U = \begin{bmatrix} u_{1} \\ \hline u_{N} \end{bmatrix}$ x
$V= (\begin{bmatrix} v_{1} \\ \hline \hline v_{M} \end{bmatrix})^T$ 

where,

- N = Number of users
- M = Number of movies
- $U$ = user embedding vector 
- $V$ = movie embedding vector

*source: [Link](https://machinelearningmastery.com/introduction-to-matrix-decompositions-for-machine-learning/)*


# Objective

To create a matrix factorization model, the inputs $U$ and $V$ has to be a sparse matrix. This is because generally in a user-movie model, not all the movies are rated by the user. The general steps to build a MF model is detailed below:

 - Create and initialise both $U$ and $V$ sparse tensor matrix with random variables 

 - Select the loss function to approximate the UT

 - Add regularization function (L1 or L2) to reduce overfitting of outlier features

 -  Perform user recommendation

In [0]:
# Build the sparse ratings matrix for our datasets

def build_rating_sparse_tensor(ratings_df):

  return tf.SparseTensor(indices=ratings[['user_id', 'movie_id']].values, values=np.array(ratings['rating']), dense_shape=(users.shape[0], movies.shape[0]))

 - Select the loss function to approximate the UV

In [0]:
def mse_error(sparse_matrix_UV, vectorU, vectorV):
    
  # Matmul to find the dot product of U and V for approximation
  #**tf.gather_nd() needs GPU runtime so that it does not check the index bound. CPU does check and will cause error**#
  predictions = tf.gather_nd(tf.matmul(vectorU, vectorV, transpose_b=True),sparse_matrix_UV.indices)
  
  # Finding the MSE between A and U*V_transpose
  return tf.losses.mean_squared_error(sparse_matrix_UV.values, predictions)

In [0]:
train_ratings, test_ratings = split_dataframe(ratings)

# initialise a U and V matrix

fakeA = tf.Variable(tf.random.normal([build_rating_sparse_tensor(test_ratings).dense_shape[0], 3]))
fakeB = tf.Variable(tf.random.normal([build_rating_sparse_tensor(test_ratings).dense_shape[1], 3]))
gatherPredictions = tf.gather(tf.matmul(fakeA, fakeB, transpose_b=True), [942, 2])

In [0]:
# Creating the training class

class Model(object):

  def __init__(self, embedding_vars, loss, metrics=None):

    self.embedding_vars = embedding_vars
    self.loss = loss
    self.metrics = metrics
    self.embeddings = {k: None for k in embedding_vars}
    self.session = None

  def embeddings(self):

    return self.embeddings

  def train(self, num_iterations=100, learning_rate=1.0, plot_results=True, optimizer=tf.train.GradientDescentOptimizer):

    with self.loss.graph.as_default():

      # Minimize loss function
      train_op = optimizer(learning_rate).minimize(self.loss)

      # Initialise the operation
      local_init_op = tf.group(tf.variables_initializer(optimizer(learning_rate).variables()), tf.local_variables_initializer())
      
      if self.session is None:
        self.session = tf.Session()

        with self.session.as_default():
          self.session.run([tf.global_variables_initializer(), tf.tables_initializer()])

    with self.session.as_default():
      local_init_op.run()
      iterations = []
      metrics = {}
      metrics_vals = {}

      # Train and append results.
      for i in range(num_iterations + 1):
        _, results = self.session.run((train_op, metrics))
        if i == num_iterations:

          for metric_val, result in zip(metrics_vals, results):

            # Embeddings are u and k respectively
            for k, v in result.items():
              metric_val[k].append(v)

      for k, v in self.embedding_vars.items():
        self.embeddings[k] = v.eval()

In [0]:
# Compute the embedding scores
# Once the model is trained, the embeddings can be inspected to understand the learned features between the users and the matrix
# Given the feedback matrix A:(MxN), the model can learn the embeddings, U (Mxd) and V(Nxd) such that the dot product of U and V_transpose will be the approximated A matrix.

def userRecommendation(model, userID):

  # Upon getting the user and movie embedding vector, the score recreates the feedback matrix. 
  userEmbedding = model.embeddings['user_id'][userID]
  movieEmbedding = model.embeddings['movie_id']

  scores = userEmbedding.dot(movieEmbedding.T)

  df = pd.DataFrame({ 
      'SCORE': list(scores),
      'MOVIE': movies['movie_id'],
      'TITLE': movies['title'],})
  
  return display.display(df.sort_values('SCORE', ascending=False).head(5)), df

- Add regularization function L1 to reduce overfitting of outlier features

In [0]:
# One of the common issue with training only with observed pairs is the problem of overfitting it. To solve this, L1 and L2 regularization can be used. These regularizations are basically penalty terms to reduce overfitting

def build_model(ratings, embedding_dim=3, regularization_coeff=0.1, init_stddev=0.1):

  # Split the training and test datasets
  train_ratings, test_ratings = split_dataframe(ratings)

  # Creating a random variable vector for the embeddings
  vectorU = tf.Variable(tf.random.normal([build_rating_sparse_tensor(train_ratings).dense_shape[0], embedding_dim], stddev=init_stddev))
  vectorV = tf.Variable(tf.random.normal([build_rating_sparse_tensor(train_ratings).dense_shape[1], embedding_dim], stddev=init_stddev))
        
  # Computing the mean squared error for feedback matrix approximation
  train_loss = mse_error(build_rating_sparse_tensor(train_ratings), vectorU, vectorV)
  test_loss = mse_error(build_rating_sparse_tensor(test_ratings), vectorU, vectorV)

  # Buiding the cost function for L1 regularization. 
  feedbackLoss = train_loss + (regularization_coeff * (tf.math.reduce_sum(tf.math.abs(vectorU))/vectorU.shape[0].value + 
                                                       tf.math.reduce_sum(tf.math.abs(vectorV))/vectorV.shape[0].value))
  loss_components = {'loss': feedbackLoss}

  embeddings = {"user_id": vectorU, "movie_id": vectorV}

  return Model(embeddings, feedbackLoss, [loss_components])


- Perform user recommendation

In [0]:
model = build_model(ratings, embedding_dim=5, regularization_coeff=0.1, init_stddev=0.1)
model.train(num_iterations=2000, learning_rate=1.)

In [132]:
# Let's try with user id number 500 and see what the model recommends the user to watch

user = 500
print('Top 5 movie recommendation for User {}: '.format(user))
df2, df_F = userRecommendation(model, user)

Top 5 movie recommendation for User 500: 


,SCORE,MOVIE,TITLE
408,4.389266,409,Jack (1996)
483,4.338067,484,"Maltese Falcon, The (1941)"
169,4.323239,170,Cinema Paradiso (1988)
114,4.285742,115,"Haunted World of Edward D. Wood Jr., The (1995)"
318,4.284071,319,Everyone Says I Love You (1996)
